In [1]:
import pandas as pd
import csv
import re
import matplotlib, seaborn, wordcloud, nltk
import numpy as np
import requests
import json
import os
from tqdm import tqdm

In [2]:
from google.colab import userdata
from google.colab import drive

In [ ]:
pip install google-api-python-client

In [3]:
from googleapiclient.discovery import build

In [4]:
youtube_api = userdata.get("YouTubeAPI")

In [5]:
youtube = build("youtube", "v3", developerKey=youtube_api)

#### Grab and save YouTube comments from each video

In [6]:
# URLS
youtube_beyonce_url = "https://www.youtube.com/watch?v=SDPITj1wlkg"
youtube_kendrick_lamar_url = "https://www.youtube.com/watch?v=KDorKy-13ak"

In [7]:
# Extract video IDs
videoID_beyonce = youtube_beyonce_url.split("v=")[1]
videoID_kendrick = youtube_kendrick_lamar_url.split("v=")[1]

In [8]:
def get_YT_comments(video_id):
    comments_data = []
    next_page_token = None

    while True:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100  # optional: speeds up data retrieval in larger chunks
        ).execute()

        for item in response['items']:
            snippet = item['snippet']['topLevelComment']['snippet']
            comment_info = {
                'video_id': video_id,
                'comment_id': item['id'],
                'comment': snippet.get('textDisplay'),
                'like_count': snippet.get('likeCount'),
                'reply_count': item['snippet'].get('totalReplyCount'),
                'published_at': snippet.get('publishedAt')
            }
            comments_data.append(comment_info)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return comments_data

In [ ]:
'''
# Get YouTube comments
def get_YT_comments(video_id):
    comments = []
    next_page_token = None

    while True:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText"
        ).execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return comments
    '''

In [9]:
# Beyonce Superbowl Performance
comments_beyonce = get_YT_comments(videoID_beyonce)
# Kendrick Lamar Performance
comments_kendrick = get_YT_comments(videoID_kendrick)

In [ ]:
print(comments_kendrick[:5])

['ight im back here again', 'Today is 2 months ago....thats 60 million vieuws in a month', 'I think I watched this over 250 times.', 'THEY NOT LIKE US 😂😂😂✌✌', 'I keep coming back here, it really is such a phenomenal performance by K-Dot and the rest of the talented performers.']


In [10]:
# Put comments in data frame
beyonce_sentiment = pd.DataFrame(comments_beyonce)
kendrick_sentiment = pd.DataFrame(comments_kendrick)

In [13]:
beyonce_sentiment.head()

,video_id,comment_id,comment,like_count,reply_count,published_at
0,SDPITj1wlkg,Ugx86qcgUZKL_hjtnFZ4AaABAg,😂,1,0,2025-04-10T11:26:30Z
1,SDPITj1wlkg,Ugwj4VFTwOf1tajTPKV4AaABAg,You could level a hockey rink with Bruno's smo...,0,0,2025-04-09T23:20:44Z
2,SDPITj1wlkg,UgyHm6Pd9QtTZPOLJIF4AaABAg,Decade later and iv still not seen coldplays part,0,0,2025-04-09T23:08:11Z
3,SDPITj1wlkg,UgyQicIZgml4fmDdLFh4AaABAg,Idk why but this was the reason I stopped liki...,0,0,2025-04-09T01:09:16Z
4,SDPITj1wlkg,UgxWmyG6W0-7hY2hwpp4AaABAg,Мои любимые❤,0,0,2025-04-08T22:57:57Z


In [14]:
kendrick_sentiment.head()

,video_id,comment_id,comment,like_count,reply_count,published_at
0,KDorKy-13ak,UgyHmP02M4AHLZCI3x14AaABAg,ight im back here again,205321,1000,2025-02-13T03:55:51Z
1,KDorKy-13ak,Ugxcn0WcofjCbyV2M814AaABAg,9:37 the vocals are INSANE!!❤️ she's so talented,0,0,2025-04-11T05:54:46Z
2,KDorKy-13ak,UgxkBF3px7Den7dyjud4AaABAg,SZA's voice is so incredible. not enough peopl...,0,0,2025-04-11T05:37:15Z
3,KDorKy-13ak,UgyVZ-3olu-FaeDEr8x4AaABAg,DRAKE still Missing 😆 🤣 😂,0,0,2025-04-11T04:25:18Z
4,KDorKy-13ak,UgwyytimtcFXRvaRrB54AaABAg,like this if its your 100th time here,4,0,2025-04-11T04:15:00Z


In [11]:
# Save data frames to CSV files
drive.mount('/content/drive')
kendrick_sentiment.to_csv("/content/drive/MyDrive/kendrick_comment_data.csv")
beyonce_sentiment.to_csv("/content/drive/MyDrive/beyonce_comment_data.csv")

Mounted at /content/drive


In [12]:
print(f'Kendrick Comments: {len(kendrick_sentiment)}')
print(f'Beyonce Comments: {len(beyonce_sentiment)}')

Kendrick Comments: 134831
Beyonce Comments: 27302
